In [7]:
!pip show lnn


Name: lnn
Version: 1.0
Summary: A `Neural = Symbolic` framework for weighted real-valued logic
Home-page: https://github.com/IBM/LNN
Author: IBM Research
Author-email: 
License: 
Location: /Users/alkutgalip/anaconda3/lib/python3.11/site-packages
Requires: jupyter, matplotlib, networkx, numpy, pandas, setuptools, tabulate, torch, tqdm
Required-by: 


In [3]:
pip install git+https://github.com/IBM/LNN


  Cloning https://github.com/IBM/LNN to /private/var/folders/gj/wrpyklq54y97gzjz_l71lh700000gp/T/pip-req-build-4g1aid3e
  Running command git clone --filter=blob:none --quiet https://github.com/IBM/LNN /private/var/folders/gj/wrpyklq54y97gzjz_l71lh700000gp/T/pip-req-build-4g1aid3e
  Resolved https://github.com/IBM/LNN to commit 2382067d91b5cba67ce90817925e6e70bc84f25a
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for torch>=1.13.1 from https://files.pythonhosted.org/packages/63/e4/efa1029677516b7d8231dca794ad445127263029b703f56d9986554a640c/torch-2.1.1-cp311-none-macosx_10_9_x86_64.whl.metadata


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 MB 2.5 MB/s eta 0:00:0000:0100:01
  Created wheel for lnn: filename=lnn-1.0-py3-none-any.whl size=80081 sha256=2647130b3c28cfd07c107d168dfca5cfdd09099c236e18db7b31995ca4f6eb61
  Stored in directory: /private/var/folders/gj/wrpyklq54y97gzjz_l71lh700000gp/T/pip-ephem-wheel-cache-c6il1mcl/wheels/45/a4/c1/ed27677834d9fd0b740698e027829b34975fdb2d1ebdfa3e2c
Successfully built lnn
Note: you may need to restart the kernel to use updated packages.


In [13]:
from lnn import Propositions
# Print the attributes of the module to understand its structure
#print(lnn_dict)


In [30]:
from lnn import Propositions, Fact, Implies, Model

# Rules
A, B, C, D, E = Propositions("A", "B", "C", "D", "E")
IMPLIES_A_B = Implies(A, B)
AND_C_D = And(C, D)
IMPLIES_CD_E = Implies(AND_C_D, E)
IMPLIES_E_CD = Implies(E, AND_C_D)
ROOT = And(IMPLIES_A_B, IMPLIES_CD_E, IMPLIES_E_CD)

# Knowledge
model = Model()
model.add_knowledge(ROOT)

# Data
A.add_data((0.2, 0.2))
C.add_data(Fact.TRUE)
E.add_data((0.3, 0.7))
ROOT.add_data(Fact.TRUE)

# Reasoning
model.infer()

# Print in the desired format
print(f"OPEN And: {ROOT} {'TRUE' if all(ROOT.get_data()) else 'FALSE'} {ROOT.get_data()}\n")

for prop in [B, D, E]:
    prop_data = prop.get_data()
    print(f"OPEN Proposition: {prop.state().name} {'APPROX_UNKNOWN' if any(prop_data) else 'APPROX_EXACT'} {prop_data}")


OPEN And: ((A → B) ∧ ((C ∧ D) → E) ∧ (E → (C ∧ D))) TRUE tensor([1., 1.], grad_fn=<SubBackward0>)

OPEN Proposition: APPROX_UNKNOWN APPROX_UNKNOWN tensor([0.2000, 1.0000])
OPEN Proposition: APPROX_UNKNOWN APPROX_UNKNOWN tensor([0.3000, 0.7000])
OPEN Proposition: APPROX_UNKNOWN APPROX_UNKNOWN tensor([0.3000, 0.7000])


In [31]:
query = Or(B, D, E)
query.upward()

0.800000011920929

In [32]:
# Knowledge
model.add_knowledge(query)

# Reasoning
print('***** USING INFERENCE ON MODEL *****')
model.infer()
query.print()

***** USING INFERENCE ON MODEL *****
OPEN Or: (B ∨ D ∨ E)                                 APPROX_TRUE (0.8, 1.0)



In [33]:
def query_state(node):
    round_off = lambda my_list: [float(f"{_:.1f}") for _ in my_list]
    return f"{node.state().name}: {tuple(round_off(node.get_data().tolist()))}"

query_state(query)

'APPROX_TRUE: (0.8, 1.0)'